# Agent

准备工作

In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents.tools import BaseTool, Tool
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain import hub
from langchain.agents import create_react_agent,AgentExecutor
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.environ.get('MOONSHOT_API_KEY')


USER_AGENT environment variable not set, consider setting it to identify your requests.


## 1 创建llm实例

创建llm实例,这里使用moonshot模型,因为兼容了openai的接口格式,所以可以直接使用ChatOpenAi类.使用一些本地模型如ollama,效果可能达不到预期,无法让agent达到finish的状态

In [2]:
llm = ChatOpenAI(model="moonshot-v1-32k", base_url="https://api.moonshot.cn/v1", api_key=API_KEY)

## 2 创建tool

agent允许使用tool, 这是扩展agent能力的关键,一个任务可以由一个或者多个tool完成, llm会在每次action的时候选择合适的tool去完成任务,tool执行后的结果会作为输入让模型开始下一轮的思考.tool中可以完成任何任务,甚至可以执行agent完成系列的任务.这里创建了一个简单的浏览网页的tool

In [3]:
def visit_website(url):
    print("visit website input:", url)
    loader = WebBaseLoader(url)
    docs = loader.load()
    return f"here is the content of the website: {docs[0].page_content}"


visit_website_tool = Tool(
    name="visit_website",
    func=visit_website,
    description="visit website by url",
)
tools = [visit_website_tool]

## 3 创建prompt

准备好prompt,这里使用hub提供的远程prompt, 也可以自己创建一个prompt, 需要包含的input_variables 有'agent_scratchpad', 'input', 'tool_names', 'tools'
promp将会引导llm来完成自我思考链,去执行action,直到能够得到答案为止. langchain内部为了避免llm进入死循环,设置了一个最大的尝试次数阈值

In [4]:
prompt = hub.pull("hwchase17/react")
print("react Prompt template:", prompt.template)

react Prompt template: Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## 4 创建agent

创建agent

In [5]:
agent = create_react_agent(llm,tools,prompt)


## 5 创建AgentExecutor

创建一个AgentExecutor,agent本身并没有实现调用逻辑

In [6]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


## 6 执行AgentExecutor

执行,给定 AgentExecutor一个输入,agent将会开始思考运行合适的tool并产生结果. AgentExecutor除了普通的invoke,也支持stream, astream等调用方式

In [7]:
result = agent_executor.invoke(
        {"input": "https://www.stats.gov.cn/zwfwck/sjfb/202406/t20240617_1954735.html 讲了什么东西"})
print(result)



> Entering new AgentExecutor chain...
I will visit the website mentioned in the question to understand what it is about.

Action: visit_website
Action Input: https://www.stats.gov.cn/zwfwck/sjfb/202406/t20240617_1954735.htmlvisit website input: https://www.stats.gov.cn/zwfwck/sjfb/202406/t20240617_1954735.html
here is the content of the website: 





国家统计局贸经司首席统计师袁彦解读5月份消费市场数据












首页

 

ENGLISH

 

RSS订阅

 

统计微博|统计微讯|移动客户端







当前位置 > 首页 > 数据发布与解读



国家统计局贸经司首席统计师袁彦解读5月份消费市场数据




发布时间：2024-06-17 15:00
					








5月份消费市场恢复向好 ——国家统计局贸经司首席统计师袁彦解读5月份消费市场数据 　　5月份，消费市场运行稳中有升，市场销售增长加快，升级类消费需求有效释放，线上消费占比持续提高；服务零售继续较快增长，出行文旅消费持续活跃。 　　一、消费品市场稳中有升，乡村市场销售快于城镇 　　在“五一”假期效应、“618”网络购物节促销以及消费品以旧换新政策等因素带动下，5月份，社会消费品零售总额39211亿元，同比增长3.7%，比4月份加快1.4个百分点。其中，商品零售额、餐饮收入分别增长3.6%、5%，增速分别加快1.6个、0.6个百分点。 　　随着县域商业体系建设不断推进，乡村市场销售持续快于城镇。5月份，乡村消费品零售额同比增长4.1%，增速比城镇高0.4个百分点，比4月份加快0.6个百分点；城镇消费品零售额增长3.7%，增速加快1.6个百分点。 　　二、服务消费较快增长，出行文旅消费持续活跃 　　1—5月份，服务零售额同比增长7.9%，受上年同期基数抬升等因素影响，增速比1—4月份有所回落。与商品零售额相比，服务零售